In [1]:
!pip install plotly==6.1.1 kaleido==0.2.1 --quiet
!pip install reportlab --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.0 MB/s eta 0:00:00


In [2]:
#only try the below cell when to check they are installed !!

In [3]:
import plotly
import kaleido
import reportlab

print("Plotly:", plotly.__version__)
print("Kaleido:", kaleido.__version__)
print("ReportLab imported successfully")


Plotly: 6.1.1
Kaleido: 0.2.1
ReportLab imported successfully


In [4]:
# Cell 2 — Imports and helper utilities
import os, glob, json, io, textwrap
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from reportlab.pdfgen import canvas
from IPython.display import display, HTML, clear_output


# Ensure reproducibility
pd.options.display.max_columns = 200
np.random.seed(42)

# Utility: find CSVs in /kaggle/input or current dir
def find_csvs():
    candidates = []
    kaggle_input = Path('/kaggle/input')
    if kaggle_input.exists():
        for f in kaggle_input.rglob('*.csv'):
            candidates.append(str(f))
    for f in Path('.').rglob('*.csv'):
        candidates.append(str(f))
    return sorted(set(candidates))

print("CSV files found:", find_csvs()[:10])


CSV files found: ['/kaggle/input/global-mobile-prize-2025-dataset/Global_Mobile_Prices_2025_Extended.csv']


In [5]:
# Cell 3 — Load dataset (auto-pick the most likely mobile dataset)
csvs = find_csvs()
if not csvs:
    raise FileNotFoundError("No CSV files found in /kaggle/input or notebook workdir. Upload your Global Mobile dataset first.")

# Heuristics: prefer filenames with 'mobile', 'phone', 'smart', 'global'
preferred = [c for c in csvs if any(x in c.lower() for x in ['mobile','phone','smart','global','review','price'])]
path = preferred[0] if preferred else csvs[0]
print("Auto-selected dataset:", path)

df = pd.read_csv(path, low_memory=False)
print("Shape:", df.shape)
display(df.head())

Auto-selected dataset: /kaggle/input/global-mobile-prize-2025-dataset/Global_Mobile_Prices_2025_Extended.csv
Shape: (1000, 15)


,brand,model,price_usd,ram_gb,storage_gb,camera_mp,battery_mah,display_size_inch,charging_watt,5g_support,os,processor,rating,release_month,year
0,Oppo,A98 111,855,16,128,108,6000,6.6,33,Yes,Android,Helio G99,3.8,February,2025
1,Realme,11 Pro+ 843,618,6,128,64,4500,6.9,100,Yes,Android,Tensor G4,4.4,August,2025
2,Xiaomi,Redmi Note 14 Pro 461,258,16,64,64,4000,6.8,44,Yes,Android,A18 Pro,4.1,March,2025
3,Vivo,V29e 744,837,6,512,48,4500,6.0,65,Yes,Android,Exynos 2400,4.1,August,2025
4,Apple,iPhone 16 Pro Max 927,335,12,128,200,5000,6.9,100,Yes,iOS,Dimensity 9300,3.5,February,2025


In [6]:
# Cell 4 — Improved date detection (no warnings)

import warnings
import pandas as pd
import numpy as np

def detect_column_types(df):
    dates = []
    numerics = []
    categoricals = []

    for c in df.columns:
        # Numeric columns first
        if np.issubdtype(df[c].dtype, np.number):
            numerics.append(c)
            continue

        # Try detect dates only if the column has a reasonable date pattern
        # This prevents false positives and reduces warnings
        sample_values = df[c].dropna().astype(str).head(50)

        looks_like_date = any(char.isdigit() for char in "".join(sample_values)) and \
                          any(sep in "".join(sample_values) for sep in ['-', '/', ':'])

        if looks_like_date:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                parsed = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)

            if parsed.notna().sum() > len(df) * 0.3:
                dates.append(c)
            else:
                categoricals.append(c)
        else:
            categoricals.append(c)

    return {
        'date_cols': dates,
        'numeric_cols': numerics,
        'categorical_cols': categoricals
    }

meta = detect_column_types(df)
meta


{'date_cols': [],
 'numeric_cols': ['price_usd',
  'ram_gb',
  'storage_gb',
  'camera_mp',
  'battery_mah',
  'display_size_inch',
  'charging_watt',
  'rating',
  'year'],
 'categorical_cols': ['brand',
  'model',
  '5g_support',
  'os',
  'processor',
  'release_month']}

In [7]:
# Cell 5 — EDA functions (summary, missing, top categories)
def quick_summary(df, meta):
    summary = {}
    summary['rows'], summary['cols'] = df.shape
    summary['num_cols'] = meta['numeric_cols']
    summary['cat_cols'] = meta['categorical_cols']
    summary['date_cols'] = meta['date_cols']
    summary['missing'] = df.isna().sum().sort_values(ascending=False).head(20).to_dict()
    # top categories sample
    cat_top = {}
    for c in meta['categorical_cols'][:10]:
        cat_top[c] = df[c].value_counts(dropna=False).head(10).to_dict()
    summary['cat_top'] = cat_top
    return summary

summary = quick_summary(df, meta)
import pprint; pprint.pprint(summary)

{'cat_cols': ['brand',
              'model',
              '5g_support',
              'os',
              'processor',
              'release_month'],
 'cat_top': {'5g_support': {'No': 497, 'Yes': 503},
             'brand': {'Apple': 107,
                       'Google': 116,
                       'Infinix': 105,
                       'OnePlus': 116,
                       'Oppo': 110,
                       'Realme': 105,
                       'Samsung': 105,
                       'Vivo': 122,
                       'Xiaomi': 114},
             'model': {'Galaxy S24 591': 2,
                       'Galaxy S24 Ultra 700': 2,
                       'Narzo 70 335': 2,
                       'Narzo 70 451': 2,
                       'Nord CE 4 Lite 268': 2,
                       'Note 40 Pro 443': 2,
                       'Note 40 Pro 876': 1,
                       'X100 Pro 55': 1,
                       'iPhone 14 43': 2,
                       'iPhone 14 583': 2},
           

In [8]:
# Cell 6 — Correlations and numeric analysis
def numeric_analysis(df, numeric_cols):
    stats = df[numeric_cols].describe().T
    # pairwise correlations (top correlated pairs)
    corr = df[numeric_cols].corr().abs().unstack().reset_index()
    corr.columns = ['a','b','corr']
    corr = corr[corr['a'] != corr['b']].sort_values('corr', ascending=False)
    top_pairs = corr.drop_duplicates(subset=['corr']).head(20)
    return stats, top_pairs

if meta['numeric_cols']:
    stats, top_pairs = numeric_analysis(df, meta['numeric_cols'])
    display(stats.head())
    display(top_pairs.head())
else:
    print("No numeric columns detected.")

,count,mean,std,min,25%,50%,75%,max
price_usd,1000.0,813.478,411.708367,101.0,449.25,822.0,1166.25,1499.0
ram_gb,1000.0,9.172,4.326330,4.0,6.00,8.0,12.00,16.0
storage_gb,1000.0,402.880,349.405893,64.0,128.00,256.0,512.00,1024.0
camera_mp,1000.0,83.534,62.504958,12.0,48.00,64.0,108.00,200.0
battery_mah,1000.0,5012.000,711.591429,4000.0,4500.00,5000.0,5500.00,6000.0


,a,b,corr
16,ram_gb,rating,0.060605
32,camera_mp,display_size_inch,0.057576
51,display_size_inch,charging_watt,0.053701
24,storage_gb,charging_watt,0.050683
69,rating,charging_watt,0.032803


In [9]:
# Cell 7 — Simple anomaly detection (z-score)
from scipy import stats as st

def detect_outliers_zscore(df, col, thresh=3.0):
    s = df[col].dropna()
    z = np.abs((s - s.mean())/s.std(ddof=0))
    return s[z > thresh].index.tolist()

outliers = {}
for c in meta['numeric_cols'][:8]:
    outliers[c] = detect_outliers_zscore(df, c)
outliers_summary = {k: len(v) for k,v in outliers.items()}
outliers_summary

{'price_usd': 0,
 'ram_gb': 0,
 'storage_gb': 0,
 'camera_mp': 0,
 'battery_mah': 0,
 'display_size_inch': 0,
 'charging_watt': 0,
 'rating': 0}

In [10]:
# CELL 8 — VISUALIZATION + save_plotly()


import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from pathlib import Path

# Ensure directory for saving images exists
IMG_DIR = Path("/kaggle/working/agent_outputs")
IMG_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# Helper: Save Plotly Figure
# ----------------------------
def save_plotly(fig, name):
    """
    Saves a Plotly figure as PNG using Kaleido.
    """
    filepath = IMG_DIR / f"{name}.png"
    fig.write_image(str(filepath), scale=2)
    return str(filepath)

# ----------------------------
# Improved Histogram Function
# ----------------------------
def plot_hist_numeric(col):
    data = df[col].dropna()

    # Freedman–Diaconis rule for adaptive bins
    q25, q75 = np.percentile(data, [25, 75])
    iqr = q75 - q25
    bin_width = 2 * iqr * (len(data) ** (-1/3))

    if bin_width <= 0:
        bin_width = (data.max() - data.min()) / 40

    num_bins = max(10, int((data.max() - data.min()) / bin_width))

    # Build histogram
    fig = go.Figure()

    fig.add_trace(go.Histogram(
        x=data,
        nbinsx=num_bins,
        opacity=0.85,
        marker=dict(line=dict(width=1)),
        hovertemplate="<b>Value:</b> %{x}<br><b>Count:</b> %{y}<extra></extra>"
    ))

    # Add reference lines
    mean_val = data.mean()
    median_val = data.median()

    fig.add_vline(
        x=mean_val,
        line_dash="dash",
        line_width=2,
        annotation_text=f"Mean {mean_val:.2f}",
        annotation_position="top right"
    )

    fig.add_vline(
        x=median_val,
        line_dash="dot",
        line_width=2,
        annotation_text=f"Median {median_val:.2f}",
        annotation_position="bottom right"
    )

    fig.update_layout(
        title=f"Histogram of '{col}'",
        xaxis_title=col,
        yaxis_title="Count",
        bargap=0.02,
        template="plotly_white"
    )

    return fig

# ----------------------------
# Improved Category Bar Plot
# ----------------------------
def plot_top_categories(col, topn=10):
    vc = df[col].value_counts(dropna=True).head(topn).reset_index()
    vc.columns = [col, "count"]

    fig = px.bar(vc, x=col, y="count", title=f"Top {topn} categories for '{col}'")
    fig.update_layout(template="plotly_white")

    return fig

# ----------------------------
# Test the visuals
# ----------------------------
if meta["numeric_cols"]:
    print("Displaying test histogram...")
    fig = plot_hist_numeric(meta["numeric_cols"][0])
    fig.show()

if meta["categorical_cols"]:
    print("Displaying test categorical plot...")
    fig2 = plot_top_categories(meta["categorical_cols"][0])
    fig2.show()

print("Cell 8 executed successfully.")


Displaying test histogram...


Displaying test categorical plot...


Cell 8 executed successfully.


In [11]:
# LISTEN in case when you fined you got Libraries to be CORRUPTED perform this but removing '#'

In [12]:
# !pip uninstall -y plotly kaleido
# !rm -rf /usr/local/lib/python3.11/dist-packages/plotly*
# !rm -rf /usr/local/lib/python3.11/dist-packages/~plotly*
# !rm -rf /usr/local/lib/python3.11/dist-packages/kaleido*


In [13]:
# Cell 9 — Time-series quick view (NOTE:- if date present)
def timeseries_overview(df, date_col, value_col=None, freq='M'):
    ts = pd.to_datetime(df[date_col], errors='coerce')
    tmp = df.copy()
    tmp[date_col] = ts
    tmp = tmp.dropna(subset=[date_col])
    if value_col and value_col in meta['numeric_cols']:
        agg = tmp.set_index(date_col)[value_col].resample(freq).mean()
        fig = px.line(agg, title=f'{value_col} over time ({freq})')
    else:
        agg = tmp.set_index(date_col).resample(freq).size()
        fig = px.line(agg, title=f'Counts over time ({freq})')
    return fig

if meta['date_cols']:
    fig = timeseries_overview(df, meta['date_cols'][0])
    display(fig)

In [14]:
#cell 10  This creates the folder:
from pathlib import Path
Path('/kaggle/working/agent_outputs').mkdir(parents=True, exist_ok=True)


In [15]:
#cell 10 . It defines the directory where all charts are saved
from pathlib import Path

IMG_DIR = Path('/kaggle/working/agent_outputs')
IMG_DIR.mkdir(parents=True, exist_ok=True)

def save_plotly(fig, name):
    """Save a plotly figure as a PNG using kaleido."""
    filepath = IMG_DIR / f"{name}.png"
    fig.write_image(str(filepath), scale=2)
    return str(filepath)


In [16]:
# Cell 11 — Report generator (PDF) using text + images
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch

def generate_pdf_report(report_path, title, summary_text, image_paths):
    c = canvas.Canvas(report_path, pagesize=A4)
    w, h = A4
    margin = 50
    y = h - margin
    c.setFont("Helvetica-Bold", 18)
    c.drawString(margin, y, title)
    y -= 30
    c.setFont("Helvetica", 10)
    for line in textwrap.wrap(summary_text, 120):
        c.drawString(margin, y, line)
        y -= 12
        if y < 120:
            c.showPage()
            y = h - margin
    # add images
    for img in image_paths:
        try:
            c.showPage()
            c.setFont("Helvetica-Bold", 12)
            c.drawString(margin, h - margin, f'Figure: {Path(img).name}')
            c.drawImage(str(img), margin, h/4, width=w-2*margin, preserveAspectRatio=True, mask='auto')
        except Exception as e:
            print("Failed adding image:", img, e)
    c.save()
    return report_path

# Example usage (will be used by agent)

In [17]:
#cell 11(ii) Test if the PDF generator actually works

test_pdf_path = "/kaggle/working/test_report.pdf"

# Make sure you have at least one plot saved in IMG_DIR
test_images = list(IMG_DIR.glob("*.png"))

if not test_images:
    print("No images found — run a plot first using Cell 8.")
else:
    output = generate_pdf_report(
        test_pdf_path,
        title="Test PDF Report",
        summary_text="This is a test PDF generated by Cell 10.",
        image_paths=test_images[:1]   # add first image
    )
    print("PDF generated at:", output)


No images found — run a plot first using Cell 8.


In [18]:
# Cell 12 — Simple ADK-style agent controller (no external LLM required)
# Cell 12 defines your AI agent. Cell 11 does not execute anything unless you call the function.
# it only defines the logic.  [  its only the engine  ]
# This maps user text intents to tool functions. For competition, this demonstrates "agent planning" and "tool calls".
def agent_handle(user_text):
    ut = user_text.lower()
    # Quick intent matching
    if any(x in ut for x in ['summary','overview','saar','summarize']):
        return ("summary", quick_summary(df, meta))
    if any(x in ut for x in ['missing','null','nan','missing values']):
        return ("missing", df.isna().sum().sort_values(ascending=False).head(30).to_dict())
    if any(x in ut for x in ['top categories','top brands','top models','most common']):
        # choose best categorical by number of unique values
        top_cat = sorted(meta['categorical_cols'], key=lambda c: df[c].nunique() if c in df else 0, reverse=True)[:3]
        plots = []
        for c in top_cat:
            fig = plot_top_categories(c)
            p = save_plotly(fig, f"top_{c}")
            plots.append(p)
        return ("top_categories_plots", plots)
    if any(x in ut for x in ['histogram','distribution','hist']):
        if not meta['numeric_cols']:
            return ("error", "No numeric columns detected for histogram.")
        c = meta['numeric_cols'][0]
        fig = plot_hist_numeric(c)
        p = save_plotly(fig, f"hist_{c}")
        return ("histogram", p)
    if any(x in ut for x in ['time','timeseries','trend','trend over time']):
        if not meta['date_cols']:
            return ("error", "No date column detected.")
        fig = timeseries_overview(df, meta['date_cols'][0])
        p = save_plotly(fig, f"timeseries_{meta['date_cols'][0]}")
        return ("timeseries", p)
    if any(x in ut for x in ['report','generate report','pdf','create report']):
        # Create some default plots and produce report
        imgs = []
        # a couple of charts if available
        if meta['numeric_cols']:
            imgs.append(save_plotly(plot_hist_numeric(meta['numeric_cols'][0]), f"report_hist_{meta['numeric_cols'][0]}"))
        for c in meta['categorical_cols'][:2]:
            imgs.append(save_plotly(plot_top_categories(c), f"report_top_{c}"))
        summary_text = "Auto-generated report: dataset shape = {} rows x {} cols. Numeric columns: {}. Categorical columns: {}. Missing values top: {}".format(
            df.shape[0], df.shape[1], meta['numeric_cols'][:5], meta['categorical_cols'][:5], 
            dict(df.isna().sum().sort_values(ascending=False).head(10))
        )
        out_pdf = str(IMG_DIR / "auto_report.pdf")
        generate_pdf_report(out_pdf, "ADK-Powered Data Insights Report", summary_text, imgs)
        return ("report_generated", out_pdf)
    # fallback: show help
    help_text = (
        "Agent understands commands containing keywords: summary, missing, top categories, histogram, timeseries, report.\n"
        "Examples:\n"
        " - 'Give me a summary'\n"
        " - 'Show top brands'\n"
        " - 'Create report'\n"
    )
    return ("help", help_text)

# Example interaction:
print(agent_handle("Give me a summary")[0])


summary


In [19]:
# Cell 12 — Interactive cell: ask the agent (use this interactively in the notebook)
# Replace the string with other prompts you want to test.
queries = [
    "Give me a summary",
    "Show top brands",
    "Show histogram",
    "Generate report"
]

for q in queries:
    print(">> USER:", q)
    code, payload = agent_handle(q)
    print(">> AGENT:", code)
    if isinstance(payload, str):
        print(payload)
    else:
        # pretty print for dicts and lists
        if isinstance(payload, dict):
            import pprint; pprint.pprint(payload)
        else:
            print(payload)
    print("-"*60)


>> USER: Give me a summary
>> AGENT: summary
{'cat_cols': ['brand',
              'model',
              '5g_support',
              'os',
              'processor',
              'release_month'],
 'cat_top': {'5g_support': {'No': 497, 'Yes': 503},
             'brand': {'Apple': 107,
                       'Google': 116,
                       'Infinix': 105,
                       'OnePlus': 116,
                       'Oppo': 110,
                       'Realme': 105,
                       'Samsung': 105,
                       'Vivo': 122,
                       'Xiaomi': 114},
             'model': {'Galaxy S24 591': 2,
                       'Galaxy S24 Ultra 700': 2,
                       'Narzo 70 335': 2,
                       'Narzo 70 451': 2,
                       'Nord CE 4 Lite 268': 2,
                       'Note 40 Pro 443': 2,
                       'Note 40 Pro 876': 1,
                       'X100 Pro 55': 1,
                       'iPhone 14 43': 2,
          